# Working with Items

DynamoDB provides four operations for basic create, read, update, and delete (CRUD) functionality described below. Each of these operations requires primary key of the item that you want to work with.

- `PutItem` creates an item
- `GetItem` reads an item
- `UpdateItem` updates an item
- `DeleteItem` deletes an item

In addition to the basic operations, DynamoDB also provides batch operations.

- `BatchGetItem` reads up to 100 items from one or more tables
- `BatchWriteItem` creates or deletes up to 25 items in one or more tables

To see how many capacity units are used for each operation, we can add `ReturnConsumedCapacity` parameter to one of the following. In this hands on, we always add the parameter to review consumed capacity units.

- `TOTAL` returns the total number of write capacity units consumed
- `INDEXES` returns the total number of write capacity units consumed, with subtotals for the table and any secondary indexes that were affected by the operation
- `NONE` is default and nothing returned

In [2]:
# import and get dynamodb resource
import boto3
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError
from pprint import pprint
from decimal import Decimal

dynamodb = boto3.resource('dynamodb')
starbucks = dynamodb.Table('Starbucks')

## PutItem

`PutItem` creates a new item or replaces an existing item if the same key already exists.

In [4]:
response = starbucks.put_item(
    Item={
        'StoreNumber': '2871-99023',
        'StoreName': 'Beomgye Station',
        'Brand': 'Starbucks'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '96',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 03 Oct 2020 16:14:00 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2831772805',
                                      'x-amzn-requestid': 'TBVQO8FRO1TK1FG65HERFC61M3VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'TBVQO8FRO1TK1FG65HERFC61M3VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [6]:
response = starbucks.put_item(
    Item={
        'StoreNumber': '283-11902',
        'StoreName': 'GS Building B1F',
        'Brand': 'Starbucks'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '96',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 03 Oct 2020 16:14:30 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2831772805',
                                      'x-amzn-requestid': '3A6LMF9PS7606E4CRQK1OQPT03VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '3A6LMF9PS7606E4CRQK1OQPT03VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


What if you want to only put item, not replace the existing item? In this case, you should use condition expressions you will learn quite soon. Condition expression determines which items should be modified. If the condition expression evaluates to true, the operation succeeds; otherwise, the operation fails.

## GetItem

Retrieving an item is done with `GetItem`. With `GetItem`, you must specify the entire primary key, not just part of it. You can use `ProjectionExpression` option to return only particular attributes from an item.

A `GetItem` request performs an eventually consistent read by default. You can use the `ConsistentRead` parameter to request a strongly consistent read instead. This consumes additional read capacity units, but it returns the most up-to-date version of the item.

In [8]:
response = starbucks.get_item(
    Key={
        'StoreNumber': '283-11902'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 0.5,
                      'Table': {'CapacityUnits': 0.5},
                      'TableName': 'Starbucks'},
 'Item': {'Brand': 'Starbucks',
          'StoreName': 'GS Building B1F',
          'StoreNumber': '283-11902'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '199',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sat, 03 Oct 2020 16:15:16 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '1609122443',
                                      'x-amzn-requestid': '2VG4OQC03F0HLJ0JTISALJHS9BVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': '2VG4OQC03F0HLJ0JTISALJHS9BVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [18]:
# it returns only StoreName attribute, but the consumed capacity is same
response = starbucks.get_item(
    Key={
        'StoreNumber': '283-11902'
    },
    ProjectionExpression='StoreName',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 0.5,
                      'Table': {'CapacityUnits': 0.5},
                      'TableName': 'Starbucks'},
 'Item': {'StoreName': 'GS Building B1F'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '141',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 15:06:38 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '245861484',
                                      'x-amzn-requestid': 'O0S722P0U6C4VVI5O9A1ADGR97VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'O0S722P0U6C4VVI5O9A1ADGR97VV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [19]:
# consistent read consumes 2x
response = starbucks.get_item(
    Key={
        'StoreNumber': '283-11902'
    },
    ConsistentRead=True,
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'Item': {'Brand': 'Starbucks',
          'StoreName': 'GS Building B1F',
          'StoreNumber': '283-11902'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '199',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 15:06:51 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '1203189529',
                                      'x-amzn-requestid': 'HUSN9G80SA2D7JK8B0HI0N7NNJVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'HUSN9G80SA2D7JK8B0HI0N7NNJVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


## UpdateItem

`UpdateItem` call is to update an existing item by modifying one or two attributes, but leaving the other attributes unchanged. When using `UpdateItem` action, you need to specify an update expression.

Update expression should include at least one of four update clauses.

- `SET` adds an attribute to an item or modify an existing attribute
- `REMOVE` deletes attributes
- `ADD` increases or decreases a number or insert elements into a set
- `DELETE` removes one or more elements from a set

You use an update expression to specify the attributes that you want to modify and their new values. Within the update expression, you use expression attribute values as placeholders for the actual values. For update expression and expression attribute values, these will be covered again soon.

`ReturnValues` parameter configures if `UpdateItem` call will return the item attributes as they appeared before or after they are updated. You can specify one of these options.

- `None` returns nothing, that is default
- `ALL_OLD` returns all of the attributes of the item before updating
- `UPDATED_OLD` returns only the updated attributes before updating
- `ALL_NEW` returns all of the attributes of the item after updating
- `UPDATED_NEW` returns only the updated attributes after updating

`ReturnValues` parameter can be set in `PutItem` and `DeleteItem` as well. With `PutItem` and `DeleteItem`, `ALL_OLD` is supported option due to the operations' characteristic.

In [25]:
# The key doesn't exist before, so this actually will put a new item
# ReturnValues parameter will make additional Attributes key in the result
response = starbucks.update_item(
    Key={
        'StoreNumber': '3912-3765'
    },
    UpdateExpression='SET StoreName = :store_name, Manager = :manager',
    ExpressionAttributeValues={
        ':store_name': 'Jeju Airport #0',
        ':manager': 'Dongkyun'
    },
    ReturnConsumedCapacity='INDEXES',
    ReturnValues='ALL_NEW'
)

pprint(response)

{'Attributes': {'Manager': 'Dongkyun',
                'StoreName': 'Jeju Airport #0',
                'StoreNumber': '3912-3765'},
 'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '206',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 15:15:05 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3125944622',
                                      'x-amzn-requestid': 'Q3LC8HIDITF023L7MH1534Q24VVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'Q3LC8HIDITF023L7MH1534Q24VVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [26]:
response = starbucks.update_item(
    Key={
        'StoreNumber': '3912-3765'
    },
    UpdateExpression='REMOVE Manager',
    ReturnValues='ALL_NEW',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'Attributes': {'StoreName': 'Jeju Airport #0', 'StoreNumber': '3912-3765'},
 'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '179',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 15:20:24 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '1991031691',
                                      'x-amzn-requestid': 'L7ADH32BVN7DHL062OMKIV458VVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'L7ADH32BVN7DHL062OMKIV458VVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


In [38]:
starbucks.put_item(
    Item={
        'StoreNumber': '92512-73',
        'StoreName': 'Worldcup Stadium',
        'Brand': 'Starbucks',
        'NumberOfEmployees': 5,
        'Employees': {'Sam', 'Tom', 'Donald', 'Susie', 'Jan'}
    },
    ReturnConsumedCapacity='INDEXES'
)

response = starbucks.update_item(
    Key={
        'StoreNumber': '92512-73'
    },
    UpdateExpression='ADD NumberOfEmployees :q, Employees :e',
    ExpressionAttributeValues={
        ':q': 2,
        ':e': {'Huang', 'Alex'}
    },
    ReturnValues='ALL_NEW',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'Attributes': {'Brand': 'Starbucks',
                'Employees': {'Alex',
                              'Donald',
                              'Huang',
                              'Jan',
                              'Sam',
                              'Susie',
                              'Tom'},
                'NumberOfEmployees': Decimal('7'),
                'StoreName': 'Worldcup Stadium',
                'StoreNumber': '92512-73'},
 'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '306',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 16:06:12 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32':

In [39]:
# generally we recommend using SET rather than ADD
# With SET, ADD can be implemented like this
response = starbucks.update_item(
    Key={
        'StoreNumber': '92512-73'
    },
    UpdateExpression='SET NumberOfEmployees = NumberOfEmployees + :q',
    ExpressionAttributeValues={
        ':q': 2
    },
    ReturnValues='ALL_NEW',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'Attributes': {'Brand': 'Starbucks',
                'Employees': {'Alex',
                              'Donald',
                              'Huang',
                              'Jan',
                              'Sam',
                              'Susie',
                              'Tom'},
                'NumberOfEmployees': Decimal('9'),
                'StoreName': 'Worldcup Stadium',
                'StoreNumber': '92512-73'},
 'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '306',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 16:06:33 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32':

## DeleteItem

`DeleteItem` deletes the item with the specified key. You can add condition expression to only delete an item under certain conditions.

In [40]:
response = starbucks.delete_item(
    Key={
        'StoreNumber': '283-11902'
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '96',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 16:07:09 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2831772805',
                                      'x-amzn-requestid': 'BHMSFER3GIINVA7TFE7D1IOD3VVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'BHMSFER3GIINVA7TFE7D1IOD3VVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


## Expression

In DynamoDB, you use expressions for various reasons.

- Projection expression is already covered in `GetItem` section, which is simple attribute name concatenation
- Condition expression
- Update expression is already covered in `UpdateItem` section, you can fully utilize it after reading grammar of expressions

To describe the usage of expressions, you could use basic expression grammar and the available kinds of expressions.

### Specifying Item Attributes

- Top-level attributes: use the attribute name
- Nested attributes: . (dot) for map, \[n\] for list

### Expression Attribute Names

An expression attribute name is an alternative to an actual attribute name. An expression attribute name must begin with a pound sign (#). There are some reasons why you need to use expression attribute name.

- Reserved words
- Attribute names containing dots
- Nested attributes
- Repeating attribute names

In [41]:
response = starbucks.update_item(
    Key={
        'StoreNumber': '92512-73'
    },
    UpdateExpression='SET #no_emp = NumberOfEmployees + :q',
    ExpressionAttributeNames={
        '#no_emp': 'NumberOfEmployees'
    },
    ExpressionAttributeValues={
        ':q': 2
    },
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '96',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 16:07:29 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2831772805',
                                      'x-amzn-requestid': 'BD4RKL9OF0I3VIELV2T4N1A53NVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'BD4RKL9OF0I3VIELV2T4N1A53NVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0}}


### Expression Attribute Values

Expression attribute values are substitutes for the actual values that you want to compare — values that you might not know until runtime. An expression attribute value must begin with a colon (:).

We have used expression attribute values in many examples, so skip examples for this.

### Condition Expression

For these data manipulation operations, you can specify a condition expression to determine which items should be modified. If the condition expression evaluates to true, the operation succeeds; otherwise, the operation fails.

Here are built-in functions and keywords for writing condition expressions. You can find details in [AWS Development Guide](https://docs.aws.amazon.com/amazondynamodb/latest/developerguide/Expressions.OperatorsAndFunctions.html).

- Comparison: `=`, `<>`, `<`, `<=`, `>`, `>=`, `BETWEEN`, `IN`
- Logical evaluations: `AND`, `OR`, `NOT`
- Functions
    - `attribute_exists`
    - `attribute_not_exists`
    - `attribute_type`
    - `begins_with`
    - `contains`
    - `size`

You use these functions and keywords in `ConditionExpression` parameter.

In [43]:
# it doesn't put the item if the key already exists
response = starbucks.put_item(
    Item={
        'StoreNumber': '283-11902',
        'StoreName': 'GS Building B1F',
        'Brand': 'Starbucks'
    },
    ReturnConsumedCapacity='INDEXES',
    ConditionExpression='attribute_not_exists(StoreNumber)'
)

pprint(response)

ConditionalCheckFailedException: An error occurred (ConditionalCheckFailedException) when calling the PutItem operation: The conditional request failed

In [46]:
# it deletes the item only when StoreName starts with GS, otherwise it returns an error
response = starbucks.delete_item(
    Key={
        'StoreNumber': '283-11902'
    },
    ReturnConsumedCapacity='INDEXES',
    ConditionExpression='begins_with(StoreName, :prefix)',
    ExpressionAttributeValues={
        ':prefix': 'GS'
    }
)

pprint(response)

ConditionalCheckFailedException: An error occurred (ConditionalCheckFailedException) when calling the DeleteItem operation: The conditional request failed

In [53]:
response = starbucks.update_item(
    Key={
        'StoreNumber': '92512-73'
    },
    UpdateExpression='SET MaxEmployee = :true',
    ConditionExpression='size(Employees) > :limit',
    ExpressionAttributeValues={
        ':true': True,
        ':limit': 3
    },
    ReturnValues='ALL_NEW',
    ReturnConsumedCapacity='INDEXES'
)

pprint(response)

{'Attributes': {'Brand': 'Starbucks',
                'Employees': {'Alex',
                              'Donald',
                              'Huang',
                              'Jan',
                              'Sam',
                              'Susie',
                              'Tom'},
                'MaxEmployee': True,
                'NumberOfEmployees': Decimal('11'),
                'StoreName': 'Worldcup Stadium',
                'StoreNumber': '92512-73'},
 'ConsumedCapacity': {'CapacityUnits': 1.0,
                      'Table': {'CapacityUnits': 1.0},
                      'TableName': 'Starbucks'},
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '335',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 04 Oct 2020 17:25:44 GMT',
                                      'server': 'Server',
              